In [3]:
import numpy as np
import pandas as pd
import os
import json
import pprint

In [4]:
dataset_name = 'segment'

In [5]:
input_dir = './raw'
output_dir = './processed'

inp_fname = 'segment.csv'

outp_fname = os.path.join(output_dir, f'{dataset_name}.csv')
outp_train_fname = os.path.join(output_dir, f'{dataset_name}_train.csv')
outp_test_fname = os.path.join(output_dir, f'{dataset_name}_test.csv')
outp_test_key_fname = os.path.join(output_dir, f'{dataset_name}_test_key.csv')
outp_infer_instances = os.path.join(output_dir, f'{dataset_name}_infer_req.json')

# Read Data

In [6]:
data = pd.read_csv(os.path.join(input_dir, inp_fname))
data.head()

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean,binaryClass
0,218,178,9,0.111111,0.0,0.833333,0.547722,1.111110,0.544331,59.629600,52.4444,75.22220,51.222200,-21.55560,46.7778,-25.22220,75.22220,0.318996,-2.04055,N
1,113,130,9,0.000000,0.0,0.277778,0.250924,0.333333,0.365148,0.888889,0.0000,2.55556,0.111111,-2.66667,5.0000,-2.33333,2.55556,1.000000,-2.12325,N
2,202,41,9,0.000000,0.0,0.944448,0.772202,1.111110,1.025600,123.037000,111.8890,139.77800,117.444000,-33.44440,50.2222,-16.77780,139.77800,0.199347,-2.29992,N
3,32,173,9,0.000000,0.0,1.722220,1.781590,9.000000,6.749490,43.592600,39.5556,52.88890,38.333300,-12.11110,27.8889,-15.77780,52.88890,0.266914,-1.99886,N
4,61,197,9,0.000000,0.0,1.444440,1.515350,2.611110,1.925460,49.592600,44.2222,61.55560,43.000000,-16.11110,35.8889,-19.77780,61.55560,0.302925,-2.02227,N


In [7]:
id_col = "id"
target_col = "binaryClass"

# Shuffle Data

In [8]:
# shuffle data
data = data.sample(frac=1, random_state=42)
data.head()

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean,binaryClass
1644,64,74,9,0.000000,0.0,1.333330,0.577777,1.388890,0.907407,21.14810,21.0000,26.88890,15.555600,-0.444444,17.22220,-16.77780,26.88890,0.419204,-1.55423,P
509,216,17,9,0.000000,0.0,0.666665,0.516400,1.277780,1.020170,126.14800,115.0000,141.88900,121.556000,-33.444400,47.22220,-13.77780,141.88900,0.189464,-2.34842,N
1410,242,183,9,0.000000,0.0,1.500000,0.936898,2.166670,1.798150,15.37040,12.6667,12.44440,21.000000,-8.111110,-8.77778,16.88890,21.00000,0.420244,2.07587,N
44,118,125,9,0.000000,0.0,0.333333,0.298142,0.888889,0.344265,1.14815,0.0000,3.11111,0.333333,-3.444440,5.88889,-2.44444,3.11111,1.000000,-2.17565,N
1584,217,45,9,0.111111,0.0,0.888888,0.207408,1.555560,0.829628,121.51900,111.1110,137.33300,116.111000,-31.222200,47.44440,-16.22220,137.33300,0.190912,-2.29267,N


# Insert Id Column

In [9]:
# insert Id column 
if id_col not in data.columns:
    N = data.shape[0]
    data.insert(0, id_col, np.arange(N))
    print(data.head())

      id  region-centroid-col  region-centroid-row  region-pixel-count  \
1644   0                   64                   74                   9   
509    1                  216                   17                   9   
1410   2                  242                  183                   9   
44     3                  118                  125                   9   
1584   4                  217                   45                   9   

      short-line-density-5  short-line-density-2  vedge-mean  vegde-sd  \
1644              0.000000                   0.0    1.333330  0.577777   
509               0.000000                   0.0    0.666665  0.516400   
1410              0.000000                   0.0    1.500000  0.936898   
44                0.000000                   0.0    0.333333  0.298142   
1584              0.111111                   0.0    0.888888  0.207408   

      hedge-mean  hedge-sd  ...  rawred-mean  rawblue-mean  rawgreen-mean  \
1644    1.388890  0.907407  ...  

# Save Main Data File

In [10]:
data.to_csv(outp_fname, index=False)

# Train Test Split

In [11]:
from sklearn.model_selection import train_test_split
test_size = 0.1

data_train, data_test = train_test_split(data, test_size=test_size, random_state=42)
print(data_train.shape, data_test.shape)

data_train.to_csv(outp_train_fname, index=False)
data_test.drop(columns=[target_col]).to_csv(outp_test_fname, index=False)
data_test[[id_col, target_col]].to_csv(outp_test_key_fname, index=False)

(2079, 21) (231, 21)


# JSON inference request instance

In [12]:
instance = data_test.drop(columns=[target_col]).reset_index(drop=True).loc[0].to_dict()
infer_req_instance_dict = {  "instances": [ {**instance}, ] }
pprint.pprint(infer_req_instance_dict)
   
with open(outp_infer_instances, 'w', encoding='utf8') as f:
    json.dump(infer_req_instance_dict, f, indent=2, ensure_ascii=False)

{'instances': [{'exblue-mean': 43.1111,
                'exgreen-mean': -24.2222,
                'exred-mean': -18.8889,
                'hedge-mean': 1.0,
                'hedge-sd': 0.666664,
                'hue-mean': -2.01531,
                'id': 1644.0,
                'intensity-mean': 113.185,
                'rawblue-mean': 127.556,
                'rawgreen-mean': 105.111,
                'rawred-mean': 106.889,
                'region-centroid-col': 9.0,
                'region-centroid-row': 94.0,
                'region-pixel-count': 9.0,
                'saturation-mean': 0.176789,
                'short-line-density-2': 0.0,
                'short-line-density-5': 0.0,
                'value-mean': 127.556,
                'vedge-mean': 0.611111,
                'vegde-sd': 0.551852}]}
